#  Lab 1, Exercise 2
## Introduction to Julia & Floating Point Arithmetic

## Using Packages
The Julia language includes many powerful features.  While many of the most commonly used functions and macros are available by default, other functions are only avaliable if you "import" a module.  For this exercise, we'll be using the Random and Statistics modules.  (Both Random and Statistics are part of Julia's "Base" set of modules that are distributed with Julia.  Later we'll learn how to add packages that are not included in Base)

To be able to access functions in a  module, execute `import MyModule` and then execute `MyModule.fn(x)` to call a function named `fn` with parameter `x`.
Alternatively, `using MyModule` will import all the functions that the Module module has specified for being "exported" into the current namespace.  Basically, this means you don't have to write `MyModule.` before every call to a function provided by `MyModule.`  Sometimes that is very convenient.  Other times that risks creating confusion about which module is being reference.

In [ ]:
using Random
using Statistics

The first time you execute any command, it will take a little longer for the Julia kernel to start.  The first time you import a module, Julia will parse the code in the module and compile (many of) its functions.  The next time you import the same module it will be faster, as it won't need to parse and compile the module (as long as the module hasn't changed).

## Writing Functions
It will be very useful to write your code as functions.  I strongly recommend you develop a habit of writing code in the form of functions.  A good rule of thumb is to try to keep each function to no more than one page of paper.  Julia provides multiple syntaxes for writing functions, as described [here in the Julia manual](https://docs.julialang.org/en/v1/manual/functions/index.html). (I suggest stopping before the subsection on "Operators Are Functions" for now.)

## Example Function
Consider an astronomer analyzing data from a large survey or simulation.  A common task is to compute the mean value ($m$) and sampler variance ($s^2$) of a data set ($y_i$) with $i=1...N$.  The data might be of observations of some quantity or the results of performing a Monte Carlo integration.  In principle, this seems very simple.  In practice, there can be some suprising behavior.  In this example, you will investigate some of the potential complications of performing even basic mathematical calculations.

a. Generate an array of simulated data using:

In [ ]:
"Generate a reproducible sample of N random variables from a Normal distribution with specified true mean."
function generate_sample(N::Integer, true_mean = 0)
   Random.seed!(42) # reseed the pseudo-random number generator
                    # so that results will be reproducible
   sample = true_mean .+ randn(N)
   return sample
end

The first line is a "docstring", it describes what the function below does
both for developers reading the code and for users who could get the
same information from a website with documentation automatically extracted
from docstrings by the Documenter.jl package.

The rest of the cell defines a function that takes two input parameters and returns a 1-d array of random variables.
The first parameter (`N`) is required and must be some form of an integer.
The second parameter (`true_mean`) could have any type and has a default value of zero.

Each time the function is called, it will begin by seeding a pseudo-random number generator with the value 42.
This is important so that results will be reproducible when run multiple times.
The function `randn` returns a 1-d array of standard random variables (i.e.,
drawn from a normal distribution with zero mean and unit variance) drawn
using Julia's default pseudo-random number generator.
Then the function returns the variable `sample`.

When you execute the code block above, julia parses the function, but does not compile or execute the code.  That will only happen once the function is called.  Since the last line of the cell is the end of the function, the output of the cell is the function.  For clarity, Jupyter prints just the function name.  Now let's try out using this function.

In [ ]:
jan1_2019 = 2458484.500000  # Julian Date for Jan 1, 2019
y = generate_sample(10, jan1_2019)

The first line demonstrates how to add a comment to your code and clarifies the reason for that number.
The second line calls the function generate_sample, asking it to compute 10
random variables with true mean equal to the julian date for January 1, 2019.

Run the above code.  Are the output consistent with your expectations?

INSERT RESPONSE

## Broadcasting
Look more closely at the function `generate_sample` above.
Note the syntax `.+` that tells julia the programmer wants to "broadcast" the scalar `true_mean` to have the same dimensions as the result of `randn(...)`.  What do you think would happen if you replaced this with `true_mean+randn(N)`?  Try it. How does the behavior compare to what you expected?

Restore the code in `generate_sample` and execute the cell again, so the
rest of the lab works as intended.

a. What is the advantage of julia having different syntax for arithmetic on variables with compatible dimensions from arithmetic on variables with different dimensions?

INSERT RESPONSE

## Calculating basic summary statistics
Now, we're going to generate a much larger sample of numbers and compute their
mean and standard deviations using multiple different methods.  You will compare the results.  The goal is to help you to understand when floating point arithmetic is likely to be problematic, so you can anticipate potential pitfalls that might affect your own research.

In [ ]:
num_obs = 100
y = generate_sample(num_obs, jan1_2019)
mean(y), std(y)

The final line calls the functions `mean` and `std` (that were exported by the Statistics package) to compute the mean and standard deviation of our sample.


## Finite precision arithmetic

By default, Julia uses 64 bits of memory to store each floating point value.  Often this is referred to as "double precission" (for historical reasons, although technically this is machine dependent and thus less precise) to differentiate it from "single precission" values typically stored with 32 bits.  To explore the effects of floating point arithmetic, convert the array of y values into arrays of floating point values that use fewer bits to store each value using the following code.

In [ ]:
y_32bit = Float32.(y)

Using the same mean and std function as before, compute (and report) the sample mean and sample variance for each of these arrays.  Subtract

In [ ]:
println("64bit:      mean=",mean(y), "  std=", std(y))
println("32bit:      mean=",mean(y_32bit), "  std=", std(y_32bit))
println("difference: ", mean(y)-mean(y_32bit), "     ", std(y)-std(y_32bit))

b. How large are the differences?  Are they significant relative to the true values?  Why is the difference for one quantity a larger fraction of its true value than the other?


INSERT RESPONCE

c. What lessons does this exercise illustrate that could be important when writing similar code for your research?

INSERT RESPONSE



## Computing Variances

Next, you will compute the variance of the above data using multiple algorithms and compare their relative merits.  Algebraically, the sample mean is calculated via
$m = 1/N \times \sum_{i=1}^{N} y_i$ and the sample variance can be written two ways
$$s^2 = 1/(N-1) \times \sum_{i=1}^N (y_i-m)^2 = 1/(N-1)  \times \left[ \left( \sum_{i=1}^N y_i^2 \right) - N m^2 \right]$$.
In this exercise, you will consider how to calculate the sample variance accurately and efficiently.
The following example demonstrates how to write a function, a simple for loop and access elements of an array in Julia:

In [ ]:
"Calculate mean value of an array using a simple for loop."
function mean_demo(y::Array)  # the syntax ::Array specifies that this function can only be applied if argument is an array.
   n = length(y);             # get the number of elements in the array y
   sum = zero(y[1]);          # set sum to zero.  Using zero(y[1]) makes sum have the same data type as y[1]
   for i in 1:n               # In Julia and Fortran, arrays start a 1, not 0 (like in C arrays and Python lists)
      sum = sum + y[i];
   end;                       # semi-colons are unnecessary, but can be useful when pasting code interactively
   return sum/n;              # return isn't necessary since functions return the last value by default
end

This could also be written more succinctly as

In [ ]:
mean_demo(y::Array) = sum(y)/length(y)

Indeed, Julia's function `Statistics.mean()` that is written almost identically to this.

a.  Write a function named `var_one_pass` that takes inputs similar to `mean_demo` and provides a one-pass algorithm to calculate the variance using a single loop.

In [ ]:
# INSERT CODE

b.  Write a function named `var_two_pass` take takes input similar to `mean_demo` and provides a two-pass algorithm to calculate the variance more accurately than the one pass algoritihm by using two loops over the $y_i$'s.

In [ ]:
# INSERT CODE

c.  Compare the accuracy of the results using data sets of different sizes ($N$) and values of $true_mean$.   Under what conditions do they give different results? 

INSERT RESPONCE

d.  What considerations would affect the decision of whether to use the one-pass algorithm or the two-pass algorithm?

INSERT RESPONCE

e.  Consider the online 1-pass algorithm for calculating the sample variance given below and then compare its results to the other algorithms for different data sets.

In [ ]:
"Compute the sample variance of an array using an online 1-pass algorithm"
function var_online(y::Array)
  n = length(y)
  sum1 = zero(y[1])
  mean = zero(y[1])
  M2 = zero(y[1])
  for i in 1:n
 diff_by_i = (y[i]-mean)/i
 mean = mean +diff_by_i
 M2 = M2 + (i-1)*diff_by_i*diff_by_i+(y[i]-mean)*(y[i]-mean)
  end
  variance = M2/(n-1)
end

Under what circumstance would it be a good/poor choice to use?

INSERT RESPONSE

f.  Don't forget that we should test your functions for accuracy.  Should we expect all of these functions to return the exact same value?  But how can we test functions that return floating point values?  
First, check that your functions pass my tests.  If not, is it because your function has a bug?  If so, fix your functions.  Or is there another explanation?  
It may help to look at the source code for the tests to see what it means to have "passed".

INSERT RESPONCE

In [ ]:
include("test/test2.jl");

g.  Can you suggest additional tests for such functions?

INSERT RESPONCE